In [1]:
# first, we import the relevent packages.

# for data manipulation
import pandas as pd
import datetime as dt

# to scrape the web
from bs4 import BeautifulSoup as soupy
import requests
import time
import random

# dealing with consistent ssl error
import ssl

# for maths
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# for writing dictionaries and others as files
import json

In [2]:
# set the url for the site with the states
states_url = 'https://state.1keydata.com'

In [3]:
# make a connection to the page
states_page = requests.get(states_url, verify=False)

C:\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'state.1keydata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
# verify the connection
states_page.status_code

200

In [5]:
# get the actual html content
states_readable = soupy(states_page.content, 'html')

In [6]:
# now we need to initialize a list to store the states
# we initialize a list with 50 none values
# this makes it much faster to insert the value into the list
list_of_states = [None] * 50

In [7]:
# now we find all the href tags 
subset_states_readable = states_readable.find('div', {'id' : 'colwrap'})
href_all = subset_states_readable.find_all('a', href=True)

In [8]:
# then we get the text for each of the a tag values
for index, state in enumerate(href_all):
    value = state.text
    value = value.replace(' ', '_')
    list_of_states[index] = value

In [9]:
# we use a list comprehension to create the urls
list_of_states = [['https://en.wikipedia.org/wiki/List_of_United_States_representatives_from_' + a, a] for a in list_of_states]

In [10]:
# temporary ssl fix
ssl._create_default_https_context = ssl._create_unverified_context

In [11]:
# initialize a list for our pandas dataframes
# this is more efficient than appending multiple data frames
list_of_data_frames = [None] * 50
# get each table as a pandas data frame
# store it in the list
for index, (url, state) in enumerate(list_of_states):
    sleep = 4
    try:
        table = pd.read_html(url)[0].iloc[:,[0,1,2,3]]
        table.rename({table.columns[0]: 'Representative'}, axis = 'columns', inplace = True)
        for i in range(1,4):
            if 'Democratic' in table.iloc[:,i].values:
                table.rename({table.columns[i]: 'Party'}, axis = 'columns', inplace = True)
            elif ('1st' or 'At-large') in table.iloc[:,i].values:
                table.rename({table.columns[i]: 'District'}, axis = 'columns', inplace = True)
            else:
                table.rename({table.columns[i]: 'Years'}, axis = 'columns', inplace = True)
        table['State'] = state
        table[['Start', 'End']] = table['Years'].str.split('–' or '-', expand = True)
        table = table.drop(columns = 'Years')
        table['Start'] = table['Start'].str.strip()
        table['End'] = table['End'].str.strip()
        table['Start'] = pd.to_datetime(table['Start'], errors='coerce')
        table['End'] = table['End'].replace(['Present', 'present'], 'November 28, 2021')
        table['End'] = pd.to_datetime(table['End'], errors='coerce')
        list_of_data_frames[index] = table
        time.sleep(random.uniform(0,sleep))
    except:
        time.sleep(random.uniform(0,sleep))
        pass

In [12]:
skel = pd.concat(list_of_data_frames, axis = 0, ignore_index = True)

In [13]:
skel = skel[skel['Start'].isnull() == False]

In [14]:
skel = skel[skel['End'].isnull() == False]

In [15]:
skel = skel[skel['End'].isnull() == False]

In [16]:
skel['End_Year'] = skel['End'].dt.year

In [20]:
skel['Start_Year'] = skel['Start'].dt.year

In [17]:
skel['Duration'] = skel['End'] - skel['Start']

In [18]:
skel['Duration_simple'] = pd.Series(skel['Duration']).dt.days/365

In [21]:
skel_2 = skel[(skel['Start_Year']>=1960)]

In [125]:
list_of_representatives = skel_2['Representative']

In [126]:
list_of_representatives_scrape = [x.replace(' ','_') for x in list_of_representatives]

In [127]:
set_of_representatives = set(list_of_representatives_scrape)

In [129]:
list_of_representatives_scrape = list(set_of_representatives)

In [133]:
list_of_urls = [['https://en.wikipedia.org/wiki/' + x,x] for x in list_of_representatives_scrape]

In [135]:
list_of_data = []
for index, (url, representative) in enumerate(list_of_urls):
    sleep = 4
    page = requests.get(url, verify=False)
    if page.status_code == 200:
        try:
            readable = soupy(page.content, 'html')
            table = readable.find('table', {'class':'infobox vcard'})
            DOB = table.find('span', {'class' : 'bday'}).text
#             df = pd.read_html(str(table))[0]
#             df = df.transpose()
#             df.columns = df.iloc[0]
#             df = df.reset_index()
#             df = df.drop(labels=0, axis=0)
#             Education = df['Education'].item().replace('(', '|').replace(')', '|').replace(',', '|').split('|')
            data = {'Representative' :  representative.replace('_', ' '), 'DOB': DOB}
            list_of_data = list_of_data.append(data)
            time.sleep(random.uniform(0,sleep))
        except:
            print(representative)
            time.sleep(random.uniform(0,sleep))
            pass

C:\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Randy_Feenstra


C:\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Bill_Chappell


C:\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

C:\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Chuck_Schumer


C:\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Katie_Porter


ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/J._Edward_Roush (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000018CFE34E8E0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))